In [6]:
import torch
import os
from ultralytics import YOLO
import cv2
import math 
import zipfile

In [7]:
torch.cuda.is_available()

True

In [8]:
zip_from = "dataset.zip"
zip_to = "dataset_original"

In [9]:
# Extract the zip file

with zipfile.ZipFile(zip_from, 'r') as zip_ref:
    zip_ref.extractall(zip_to)

In [10]:
# Get all the files in a directory

directories = ["test", "train", "valid"]
sub_directories = ["rock", "paper", "scissors"]

for directory in directories:
    for dir_index, sub_directory in enumerate(sub_directories):
        path = os.path.join(zip_to, directory, sub_directory)
        files = os.listdir(path)
        # iterate over all the files
        for idx, file in enumerate(files):
            # read the image
            # print(file)
            # get the extension of the file
            extension = file.split(".")[-1]
            if (extension == "jpg"):
                # rename the file with the directory name and the idx
                new_name = sub_directory + "_" + str(idx) + ".jpg"
                # rename the file
                os.rename(os.path.join(path, file), os.path.join(path, new_name))
                # create an annotation file
                annotation_file = open(os.path.join(path, new_name.split(".")[0] + ".txt"), "w")
                # write the string Each row is class x_center y_center width height format.
                # class 0 0 11
                annotation_file.write(f"{dir_index} 0 0 1 1")
                # close the file
                annotation_file.close()

In [12]:
# train a yolo model based on the dataset

model = YOLO('yolov8n.pt')

# model = YOLO('rps.yaml', task="classify")

results = model.train(data='rps.yaml', epochs=10, imgsz=640)



Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-1.12.1+cu102 CUDA:0 (NVIDIA GeForce RTX 2070, 7966MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=rps.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

In [14]:
# use the results to predict the images

# path to trained weights
weights_path = "../runs/detect/train5/weights/best.pt"

new_model = YOLO(weights_path, task="detect")